# Data Collection

In order to begin analyzing stocks we first need to collect data to analyze. We will do this by scraping a specified timeframe of stock information for a specified number of stocks.


## Getting Started
To get started, we must import some libraries:

In [4]:
import sys
!{sys.executable} -m pip install yfinance
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
print("======================")


import yfinance as yf
import pandas as pd

print ("Successfully imported libraries")

Successfully imported libraries


We will begin by specifying a list of tickers we want to scrape:

In [5]:
tickers = ["AMZN", "TSLA", "BABA", "HD", "TM", "NKE", "MCD", "LOW", "SBUX", "JD", "PDD", "BKNG", "GM", "MELI", "TJX", "NIO", "F", "LULU", "HMC", "CMG"]

From here, we have to define some options:

- Interval: This represents the frequency we want to get stock data
- Time Frame (2 options):
    - A)
        - Period: This represents a timeframe in english (1m, 5m, 1d, 1y, ...)
    - B)
        - Start Date: This is the starting date for data scraping
        - End Date: This is the ending date for data scraping

In [6]:
period = '10y'
start_date = None
end_date = None
interval = "1d"

Just a quick check to ensure variables are instantiated correctly:

In [7]:
if tickers == None:
    raise Exception("You must specify a list of tickers to scrape")

if period == None and start_date == None and end_date == None:
            raise Exception("You must specify one timeframe in order to scrape")
        
if period == None and (start_date == None or end_date == None):
    raise Exception("You must specify both ends of the timeframe in order to scrape")

if period not in ['1d','5d','1mo','3mo','6mo','1y','2y','5y','10y','ytd','max']:
    raise Exception("Please input a valid period")

if interval not in ['1m','2m','5m','15m','30m','60m','90m','1h','1d','5d','1wk','1mo','3mo']:
    raise Exception("Please input a valid time interval")

if period != None and (start_date != None or end_date != None):
    raise Exception("You can only specify one type of timeframe in order to scrape")

print("Passed!")

Passed!


## Scraping Data
Now that we have defined the scraping parameters, we can actually begin to scrape. We do this by calling the YFinance download method for each stock. However, we have to consider two cases:
- We are using period
- We are using start/end dates


In [8]:
ticker_string = ' '.join(tickers) #convert from list to space-separated string
print(f"Ticker String: \n{ticker_string}")

if period != None: #if using period
    data = yf.download(
        ticker_string,
        period = period,
        interval = interval,
        group_by = 'ticker',
        threads = True
    )
else:
    data = yf.download( #if using start/end dates
        ticker_string,
        start = start_date, 
        end = end_date,
        interval = interval,
        group_by = 'ticker',
        threads = True
    )


data = data.drop([(i, 'Close') for i in tickers], axis=1)
data = data.rename({"Adj Close": "Close"}, axis=1)


data.head()

Ticker String: 
AMZN TSLA BABA HD TM NKE MCD LOW SBUX JD PDD BKNG GM MELI TJX NIO F LULU HMC CMG
[*********************100%***********************]  20 of 20 completed


NIO                             MELI                        \
           Open High Low Close Volume       Open       High        Low   
Date                                                                     
2011-11-21  NaN  NaN NaN   NaN    NaN  81.559998  82.160004  80.000000   
2011-11-22  NaN  NaN NaN   NaN    NaN  81.870003  83.209999  80.820000   
2011-11-23  NaN  NaN NaN   NaN    NaN  83.300003  83.300003  80.050003   
2011-11-25  NaN  NaN NaN   NaN    NaN  80.559998  81.919998  80.000000   
2011-11-28  NaN  NaN NaN   NaN    NaN  81.980003  85.519997  81.980003   

                                ...        BKNG                          \
                Close   Volume  ...        Open        High         Low   
Date                            ...                                       
2011-11-21  79.517319  1040800  ...  486.970001  494.910004  474.380005   
2011-11-22  79.770126   605300  ...  472.779999  485.989990  472.160004   
2011-11-23  78.331078   727600  ...  478.000000  479.390015  464.029999   
2011-11-25  78.233856   247500  ...  462.970001  466.970001  456.269989   
2011-11-28  82.463486   573500  ...  474.510010  483.029999  468.750000   

                                       NKE                                   \
                 Close   Volume       Open       High        Low      Close   
Date                                                                          
2011-11-21  477.649994  1225300  22.905001  23.040001  22.447500  20.255486   
2011-11-22  481.070007   894800  22.797501  23.155001  22.747499  20.382282   
2011-11-23  464.529999   894900  22.682501  22.992500  22.625000  20.226578   
2011-11-25  459.179993   447800  22.665001  22.965000  22.570000  20.081989   
2011-11-28  472.500000   893900  23.010000  23.957500  22.975000  21.111885   

                      
              Volume  
Date                  
2011-11-21   7354000  
2011-11-22   7660000  
2011-11-23   7470400  
2011-11-25   3475600  
2011-11-28  15442400  

[5 rows x 100 columns]

# Adding Technical Indicators

Using the [TA-Lib](https://mrjbq7.github.io/ta-lib/) package, we can broadcast technical indicators across the time series into their own dataframe columns

In [9]:
import talib
!{sys.executable} -m pip install TA-Lib

def computeRSI(df):
    return talib.RSI(df["Close"], timeperiod=14)

def computeUltimateOscillator(df):
    return talib.ULTOSC(df["High"], df["Low"], df["Close"], timeperiod1=7, timeperiod2=14, timeperiod3=28)

def computeBollingerBands(df):
    upperband, middleband, _ = talib.BBANDS(df["Close"], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    return (upperband - middleband) / df["Close"] # need to normalize so value is not dependent on share price

def computeChaikinOscillator(df):
    return talib.ADOSC(df["High"], df["Low"], df["Close"], df["Volume"], fastperiod=3, slowperiod=10)

def computeNATR(df):
    return talib.NATR(df["High"], df["Low"], df["Close"], timeperiod=14)

def computeSMA(df, window):
    return talib.SMA(df["Close"], timeperiod=window) / df["Close"] # need to normalize so value is not dependent on share price

def computeSAR(df):
    return talib.SAR(df["High"], df["Low"], acceleration=0, maximum=0) / df["Close"] # need to normalize so value is not dependent on share price

def computeWilliamsR(df):
    return talib.WILLR(df["High"], df["Low"], df["Close"], timeperiod=14)

def computeAPO(df):
    return talib.APO(df["Close"], fastperiod=12, slowperiod=26, matype=0) / df["Close"]# need to normalize so value is not dependent on share price

def computeROC(df, window):
    return talib.ROCP(df["Close"], timeperiod=window) 


Now, we can apply these functions across the collected timeseries

In [10]:
for ticker in tickers:
    df = data[ticker]
    data[ticker, 'RSI'] = computeRSI(df)
    data[ticker, 'Ultimate'] = computeUltimateOscillator(df)
    data[ticker, 'BandRadius'] = computeBollingerBands(df)
    data[ticker, 'Chaikin'] = computeChaikinOscillator(df)
    data[ticker, 'NATR'] = computeNATR(df)
    data[ticker, 'SMA_5'] = computeSMA(df, 5)
    data[ticker, 'SMA_20'] = computeSMA(df, 20)
    data[ticker, 'SMA_100'] = computeSMA(df, 100)
    data[ticker, 'SAR'] = computeSAR(df)
    data[ticker, 'Williams%R'] = computeWilliamsR(df)
    data[ticker, 'APO'] = computeAPO(df)
    data[ticker, 'ROC_5'] = computeROC(df, 5)
    data[ticker, 'ROC_20'] = computeROC(df, 20)
    data[ticker, 'ROC_100'] = computeROC(df, 100)

data.head(100)

NIO                             MELI                        \
           Open High Low Close Volume       Open       High        Low   
Date                                                                     
2011-11-21  NaN  NaN NaN   NaN    NaN  81.559998  82.160004  80.000000   
2011-11-22  NaN  NaN NaN   NaN    NaN  81.870003  83.209999  80.820000   
2011-11-23  NaN  NaN NaN   NaN    NaN  83.300003  83.300003  80.050003   
2011-11-25  NaN  NaN NaN   NaN    NaN  80.559998  81.919998  80.000000   
2011-11-28  NaN  NaN NaN   NaN    NaN  81.980003  85.519997  81.980003   
...         ...  ...  ..   ...    ...        ...        ...        ...   
2012-04-10  NaN  NaN NaN   NaN    NaN  96.599998  98.650002  92.169998   
2012-04-11  NaN  NaN NaN   NaN    NaN  94.430000  96.330002  93.750000   
2012-04-12  NaN  NaN NaN   NaN    NaN  94.510002  97.779999  93.980003   
2012-04-13  NaN  NaN NaN   NaN    NaN  97.389999  98.110001  95.800003   
2012-04-16  NaN  NaN NaN   NaN    NaN  97.059998  97.070000  91.860001   

                                ...       CMG                                \
                Close   Volume  ...      NATR     SMA_5    SMA_20   SMA_100   
Date                            ...                                           
2011-11-21  79.517319  1040800  ...       NaN       NaN       NaN       NaN   
2011-11-22  79.770126   605300  ...       NaN       NaN       NaN       NaN   
2011-11-23  78.331078   727600  ...       NaN       NaN       NaN       NaN   
2011-11-25  78.233856   247500  ...       NaN       NaN       NaN       NaN   
2011-11-28  82.463486   573500  ...       NaN  0.985114       NaN       NaN   
...               ...      ...  ...       ...       ...       ...       ...   
2012-04-10  90.885376   401700  ...  1.746420  1.011574  0.999108       NaN   
2012-04-11  92.843781   290300  ...  1.768175  0.997308  0.987505       NaN   
2012-04-12  95.094460   236800  ...  1.781078  0.980467  0.967268       NaN   
2012-04-13  94.120140   248200  ...  1.843905  0.971485  0.955282       NaN   
2012-04-16  89.609001   437500  ...  1.960223  0.992789  0.976040  0.856738   

                                                                       
                 SAR Williams%R       APO     ROC_5    ROC_20 ROC_100  
Date                                                                   
2011-11-21       NaN        NaN       NaN       NaN       NaN     NaN  
2011-11-22  0.981516        NaN       NaN       NaN       NaN     NaN  
2011-11-23  1.039175        NaN       NaN       NaN       NaN     NaN  
2011-11-25  1.038138        NaN       NaN       NaN       NaN     NaN  
2011-11-28  0.969149        NaN       NaN       NaN       NaN     NaN  
...              ...        ...       ...       ...       ...     ...  
2012-04-10  0.720081 -69.153142  0.022305 -0.004254  0.048048     NaN  
2012-04-11  0.709790 -35.277636  0.020695  0.000947  0.056518     NaN  
2012-04-12  0.692489  -1.383300  0.017753  0.038797  0.083050     NaN  
2012-04-13  0.681199  -6.393862  0.016846  0.036284  0.081851     NaN  
2012-04-16  0.694252 -32.864448  0.016468  0.013629  0.051617     NaN  

[100 rows x 380 columns]

At this point, we have a 3D nested dataframe, split by tickers and then by columns. To put this data in CSV files, we will create a CSV file for each ticker containing the relevant information.

In [11]:
import os

for ticker in tickers:
    print(f"Saving data for {ticker}")
    
    tickerdata = data[ticker].dropna() #remove nan values
    if not os.path.exists("data"):
        os.mkdir("data")
    tickerdata.to_csv(f'./data/{ticker}.csv') #write to csv file

print("======================")
print("Saved stocks to CSV files. Take a look inside /data for them.")

Saving data for AMZN
Saving data for TSLA
Saving data for BABA
Saving data for HD
Saving data for TM
Saving data for NKE
Saving data for MCD
Saving data for LOW
Saving data for SBUX
Saving data for JD
Saving data for PDD
Saving data for BKNG
Saving data for GM
Saving data for MELI
Saving data for TJX
Saving data for NIO
Saving data for F
Saving data for LULU
Saving data for HMC
Saving data for CMG
Saved stocks to CSV files. Take a look inside /data for them.
